In [ ]:
#8-4 Consolidar 
#Reconhecimento de Textos - OCR
#Tensorflow, Learnig machine, Deep learnig, Mnisp, Kaglle

#Importando as bibliotecas
import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import zipfile
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import imutils
#from google.colab.patches import cv2_imshow
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
#Carregando as bases de dados
#Conjunto MNIST 0-9
from tensorflow.keras.datasets import mnist

(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

train_data.shape, test_data.shape
train_labels.shape, test_labels.shape

digitos_data = np.vstack([train_data, test_data])
digitos_labels = np.hstack([train_labels, test_labels])

np.random.randint(0, digitos_data.shape[0])

indice = np.random.randint(0, digitos_data.shape[0])
plt.imshow(digitos_data[indice], cmap='gray')
plt.title('Classe: ' + str(digitos_labels[indice]));

sns.countplot(digitos_labels);

In [ ]:
#Conjunto Kaggle A-Z

#!wget https://iaexpert.academy/arquivos/alfabeto_A-Z.zip
#!wget https://drive.google.com/file/d/1BWkPbX5e76-x4yLxAorfgoSicq5ectx0/view?usp=sharing

#zip_object = zipfile.ZipFile(file='/content/drive/MyDrive/alfabeto_A-Z.zip', mode = 'r')
#zip_object.extractall('./')
#zip_object.close()

dataset_az = pd.read_csv("D:/Dados/alfabeto_A-Z/A_Z Handwritten Data.csv").astype('float32')

alfabeto_data = dataset_az.drop('0', axis = 1)
alfabeto_labels = dataset_az['0']

alfabeto_labels

alfabeto_data = np.reshape(alfabeto_data.values, (alfabeto_data.shape[0], 28, 28))

indice = np.random.randint(0, alfabeto_data.shape[0])
plt.imshow(alfabeto_data[indice], cmap = 'gray')
plt.title('Classe ' + str(alfabeto_labels[indice]));

In [ ]:
#Junção das bases de dados

digitos_labels, np.unique(digitos_labels)

alfabeto_labels, np.unique(alfabeto_labels)

alfabeto_labels += 10

data = np.vstack([alfabeto_data, digitos_data])
labels = np.hstack([alfabeto_labels, digitos_labels])

data = np.array(data, dtype='float32')

data = np.expand_dims(data, axis = -1)

In [ ]:
#Pré-processamento dos dados

data[0].min(), data[0].max()
data /= 255.0
np.unique(labels)

le = LabelBinarizer()
labels = le.fit_transform(labels)

classes_total = labels.sum(axis=0)

plt.imshow(data[30000].reshape(28,28), cmap='gray')
plt.title(str(labels[30000]));

classes_peso = {}
for i in range(0, len(classes_total)):
  #print(i)
  classes_peso[i] = classes_total.max() / classes_total[i]

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state = 1)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
augmentation = ImageDataGenerator(rotation_range=10, zoom_range=0.05, width_shift_range=0.1,
                                  height_shift_range=0.1, horizontal_flip = False)

In [ ]:
#Criação da estrutura da rede neural

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

rede_neural = Sequential()

rede_neural.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
rede_neural.add(MaxPool2D(pool_size=(2,2)))

rede_neural.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
rede_neural.add(MaxPool2D(pool_size=(2,2)))

rede_neural.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='valid'))
rede_neural.add(MaxPool2D(pool_size=(2,2)))

rede_neural.add(Flatten())

rede_neural.add(Dense(64, activation='relu'))
rede_neural.add(Dense(128, activation='relu'))

rede_neural.add(Dense(36, activation='softmax'))

rede_neural.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

rede_neural.summary()

nomes_labels = '0123456789'
nomes_labels += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
nomes_labels = [l for l in nomes_labels]

In [ ]:
#Treinamento da rede neural

arquivo_modelo = 'manuscrito.keras'
epochs = 20
batch_size = 128

checkpointer = ModelCheckpoint(arquivo_modelo, monitor='val_loss', verbose=1, save_best_only=True)

len(X_train) // batch_size

history = rede_neural.fit(augmentation.flow(X_train, y_train, batch_size=batch_size),
                                            validation_data = (X_test, y_test),
                                            steps_per_epoch = len(X_train) // batch_size,
                                            epochs = epochs,
                                            class_weight = classes_peso,
                                            verbose=1, callbacks=[checkpointer])

In [ ]:
#Avaliação da rede neural
previsoes = rede_neural.predict(X_test, batch_size=batch_size)

len(previsoes[0])

np.argmax(previsoes[0])

rede_neural.evaluate(X_test, y_test)

history.history.keys()

plt.plot(history.history['val_loss']);

plt.plot(history.history['val_accuracy']);

In [ ]:
#Conectando com o Drive e acessando os arquivos
#Carregando a rede neural

#from google.colab import drive
#drive.mount('/content/drive')
# Realize o dowload da pasta Material_complementar_reconhecimento_texto.zip do Google Sala de Aula e transira-a para o seu Google Drive
# Localize o caminho da pasta no menu Arquivos, no menu lateral esquerdo
#path = "/content/drive/MyDrive/Material_complementar_reconhecimento_texto.zip"
#zip_object = zipfile.ZipFile(file=path, mode="r")
#zip_object.extractall("./")
#import tensorflow as tf

rede_neural = load_model("D:/Dados/Material_complementar_reconhecimento_texto/modelos/rede_neural.h5")
#rede_neural = tf.keras.models.load_model("D:/Dados/Material_complementar_reconhecimento_texto/modelos/rede_neural")
rede_neural.summary()

In [ ]:
#Carregando a imagem de teste

from PIL import Image
import matplotlib.pyplot as plt

img = cv2.imread('D:/Dados/Material_complementar_reconhecimento_texto/imagens/manuscrito.jpg')
#cv2_imshow(img)
plt.imshow(img)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()


gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#cv2_imshow(gray)
plt.imshow(gray)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()


In [ ]:
#Pré-processamento da imagem
desfoque = cv2.GaussianBlur(gray, (3,3), 0)
#cv2_imshow(desfoque)
plt.imshow(desfoque)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()

In [ ]:
adapt_media = cv2.adaptiveThreshold(desfoque, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 9)
#cv2_imshow(adapt_media)
plt.imshow(adapt_media)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()

In [ ]:
inv = 255 - adapt_media
#cv2_imshow(inv)
plt.imshow(adapt_media)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()

In [ ]:
dilatado = cv2.dilate(inv, np.ones((3,3)))
#cv2_imshow(dilatado)
plt.imshow(dilatado)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()

In [ ]:
bordas = cv2.Canny(desfoque, 40, 150)
#cv2_imshow(bordas)
plt.imshow(dilatado)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()

In [ ]:
dilatado = cv2.dilate(bordas, np.ones((3,3)))
#cv2_imshow(dilatado)
plt.imshow(dilatado)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()

In [ ]:
#Detecção de contornos

def encontrar_contornos(img):
  conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  conts = imutils.grab_contours(conts)
  conts = sort_contours(conts, method='left-to-right')[0]
  return conts

conts = encontrar_contornos(dilatado.copy())

l_min, l_max = 4, 160
a_min, a_max = 14, 140

caracteres = []
img_cp = img.copy()
for c in conts:
  #print(c)
  (x, y, w, h) = cv2.boundingRect(c)
  #print(x, y, w, h)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    roi = gray[y:y+ h, x:x + w]
    #cv2_imshow(roi)
    thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    #cv2_imshow(thresh)
    plt.imshow(thresh)
    plt.axis('off')  # Desativar eixos para uma visualização mais limpa
    plt.show()
    cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255, 100, 0), 2)
#cv2_imshow(img_cp)
plt.imshow(img_cp)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()

In [ ]:
#Processando os caracteres detectados
#Extração ROI
def extra_roi(img):
  roi = img[y:y + h, x:x + w]
  return roi

#Limiarização
def limiarizacao(img):
  thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  return thresh


In [ ]:
#Redimensionamento

def redimensiona_img(img, l, a):
  if l > a:
    redimensionada = imutils.resize(img, width=28)
  else:
    redimensionada = imutils.resize(img, height=28)

  (a, l) = redimensionada.shape
  dX = int(max(0, 28 - l) / 2.0)
  dY = int(max(0, 28 - a) / 2.0)

  preenchida = cv2.copyMakeBorder(redimensionada, top=dY, bottom=dY, right=dX, left=dX, borderType=cv2.BORDER_CONSTANT, value=(0,0,0))
  preenchida = cv2.resize(preenchida, (28, 28))
  return preenchida

  (x, y, w, h) = cv2.boundingRect(conts[6])
print(x, y, w, h)
img_teste = limiarizacao(gray[y:y+h, x:x+w])
#cv2_imshow(img_teste)
plt.imshow(img_teste)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()
(a, l) = img_teste.shape
print(a, l)
img_teste2 = redimensiona_img(img_teste, l, a)
#Cv2_imshow(img_teste2)
plt.imshow(img_teste2)
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()
img_teste2.shape

#cv2_imshow(cv2.resize(img_teste, (28,28)))
plt.imshow(cv2.resize(img_teste, (28,28)))
plt.axis('off')  # Desativar eixos para uma visualização mais limpa
plt.show()

In [ ]:
#Normalização

def normalizacao(img):
  img = img.astype('float32') / 255.0
  img = np.expand_dims(img, axis=-1)
  return img

  img_teste2.shape, normalizacao(img_teste2).shape

In [ ]:
#Processamento das detecções
def processa_caixa(gray, x, y, w, h):
  roi = extra_roi(gray)
  limiar = limiarizacao(roi)
  (a, l) = limiar.shape
  redimensionada = redimensiona_img(limiar, l, a)
  #cv2_imshow(redimensionada)
  plt.imshow(redimensionada)
  plt.axis('off')  # Desativar eixos para uma visualização mais limpa
  plt.show()
  normalizada = normalizacao(redimensionada)
  caracteres.append((normalizada, (x, y, w, h)))


for c in conts:
  #print(c)
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)

caixas = [caixa[1] for caixa in caracteres]
caracteres = np.array([c[0] for c in caracteres], dtype='float32')

In [ ]:
#Reconhecimento dos caracteres
numeros = "0123456789"
letras = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
lista_caracteres = numeros + letras
lista_caracteres = [l for l in lista_caracteres]

caracteres[0].shape

caracteres.shape

previsoes = rede_neural.predict(caracteres)

previsoes.shape

img_cp = img.copy()
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i = np.argmax(previsoes)
  #print(i)
  probabilidade = previsoes[i]
  #print(probabilidade)
  caractere = lista_caracteres[i]
  #print(caractere)

  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255,100,0), 2)
  cv2.putText(img_cp, caractere, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,0,255), 2)
  print(caractere, ' -> ', probabilidade * 100)

  #v2_imshow(img_cp)
  plt.imshow(img_cp)
  plt.axis('off')  # Desativar eixos para uma visualização mais limpa
  plt.show()

def extrai_roi(img, margem=2):
  roi = img[y - margem:y + h + margem, x - margem:x + w + margem]
  return roi


conts = encontrar_contornos(dilatado.copy())
caracteres = []
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)


caixas = [b[1] for b in caracteres]
caracteres = np.array([c[0] for c in caracteres], dtype="float32")
previsoes = rede_neural.predict(caracteres)

img_cp = img.copy()
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i = np.argmax(previsoes)
  probabilidade = previsoes[i]
  caractere = lista_caracteres[i]

  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255,100,0), 2)
  cv2.putText(img_cp, caractere, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,0,255), 2)
  print(caractere, ' -> ', probabilidade * 100)

  #cv2_imshow(img_cp)
  plt.imshow(img_cp)
  plt.axis('off')  # Desativar eixos para uma visualização mais limpa
  plt.show()